# Clustering 01

In this notebook, we need to find which clusters correspond with which cortex layer in the primary data. 

In [13]:
import pandas as pd 
import numpy as np
import sklearn as sk
import hdbscan
import dask.dataframe as da
import dask
from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar = pbar.register()

Let's read in the data, and then perform a sample run of density-based clustering, and visualize our results with a UMAP projection onto $\mathbb{R}^2$

In [14]:
# organoid = da.read_csv('../data/processed/organoid.csv')
primary = pd.read_csv('../data/processed/primary.csv', nrows=10000)

## Clustering

We begin by using HDBSCAN, a density based clustering method that makes two important assumptions: our clusters are not Gaussian balls, and we don't know the number of true clusters a priori. Especially since we cannot reasonably visualize our data pairwise (2^16k plots).

In [18]:
def cluster(data):
    clusterer = hdbscan.HDBSCAN(core_dist_n_jobs=-1)
    return clusterer.fit(primary)

In [19]:
prim_cluster = cluster(primary)

Let's look at what clusters were found by HDBSCAN

In [ ]:
from sklearn.cluster import KMeans

## Cluster visualization

Now that we've performed clustering using HDBSCAN, let's visualize the 2D projection of the data using UMAP